In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.preprocessing import image
from PIL import Image
import io
from datetime import datetime
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

In [2]:
# Pay attemtion to the path 
base_dir = '../real_vs_fake/real-vs-fake'
#|--project
#|-----TechLabs-DL-2024
#|----------model.ipynb
#|----------README.md
#|-----real-vs-fake
#|----------test
#|----------train
#|----------valid

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

In [3]:
batch_size = 250
img_size = (150,150)
# sample_count = 102041 not needed anymore


train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['real', 'fake'],
    shuffle=True,
    subset=None,
    seed=42
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['real', 'fake'],
    shuffle=True,
    subset=None,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['real', 'fake'],
    shuffle=False
)

# train_generator.samples = 2 * sample_count  # causing errors in training
#train_generator.batch_size = batch_size

Found 102041 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
# Load VGG16
base_model = tf.keras.applications.VGG16(
    weights='imagenet', 
    include_top=False,  # Exclude the fully connected layer
    input_shape=(150, 150, 3)  # Specify the input shape
)

In [5]:
# Freeze the first 10 layers of the base model
for layer in base_model.layers[:8]:
    layer.trainable = False

# Leave the remaining layers trainable
for layer in base_model.layers[8:]:
    layer.trainable = True

In [6]:
# Add custom layers on top of the base model
model = models.Sequential([
    base_model,  # Add the base model as the first layer
    layers.GlobalAveragePooling2D(),  # Reduces dimensions without further convolutions
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
# adding early stopping to not miss best model

early_stopping = EarlyStopping(
    monitor='val_loss',     # Continiously validation loss
    mode='min',             # Stops when val_loss is minimized
    patience=3,             # Waits 3 epochs id no improvement
    restore_best_weights=True  # Restores best model weights when training stops
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # number of batches per epoch
    epochs=100,  # high number of epochs is ok due to early stopping
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    callbacks=[early_stopping]   # EarlyStopping callback 
)

Epoch 1/100


2024-09-11 21:39:40.164494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


408/408 [==============================] - ETA: 0s - loss: 0.7335 - accuracy: 0.5044 

2024-09-12 00:23:06.991485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


408/408 [==============================] - 10702s 26s/step - loss: 0.7335 - accuracy: 0.5044 - val_loss: 0.6931 - val_accuracy: 0.5002
Epoch 2/100
408/408 [==============================] - 10768s 26s/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/100
408/408 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5010 

In [ ]:
# Evaluate on the Test Set
print("Evaluating on Test Set:")
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

Y_pred_test = model.predict(test_generator)
y_pred_test = np.round(Y_pred_test).astype(int)

print('Confusion Matrix - Test Set')
print(confusion_matrix(test_generator.classes, y_pred_test))

print('Classification Report - Test Set')
print(classification_report(test_generator.classes, y_pred_test, target_names=['Real', 'Fake']))

# Evaluate on the Validation Set
print("\nEvaluating on Validation Set:")
val_loss, val_acc = model.evaluate(valid_generator, steps=valid_generator.samples // valid_generator.batch_size)
print(f"Validation Accuracy: {val_acc:.2f}")

Y_pred_val = model.predict(valid_generator)
y_pred_val = np.round(Y_pred_val).astype(int)

print('Confusion Matrix - Validation Set')
print(confusion_matrix(valid_generator.classes, y_pred_val))

print('Classification Report - Validation Set')
print(classification_report(valid_generator.classes, y_pred_val, target_names=['Real', 'Fake']))

In [ ]:
model.save('pretrained_trainable_VGG16.h5')  # HDF5 format

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(7)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
history_df

In [ ]:
# lets add model history to CSV so that we can make a cool graph out of it later
# Pls change layer name :)
history_df.to_csv("trainable_VGG16_model_history.csv", index = True)